In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [20]:
options = Options()
options.add_argument('--headless')
navegador = webdriver.Edge(options=options)
navegador.get('https://app.solarz.com.br/login?logout')
# Para abrir o webdriver, deve selecionar o interpreter >> UFPI (Python 3.9.2)

# Inserção de dados na tela de login
login = navegador.find_element(By.NAME, 'username')
login.send_keys('contato@energiacruze.com')
password = navegador.find_element(By.NAME, 'password')
password.send_keys('Cruze2113!') 
botao_entrar = navegador.find_element(By.CLASS_NAME, 'btnEntrar')
botao_entrar.send_keys(Keys.ENTER) # Pressionar o Enter no botão de "ENTRAR"


In [ ]:
# IMPLEMENTAÇÃO DE BOT PARA SUPERVISÃO SEMANAL DE GERAÇÃO DAS USINAS CADASTRADAS

sleep(1)
navegador.get('https://app.solarz.com.br/integrador/usinas') # Requisição para visualização das usinas
sleep(10)
list_usinas = BeautifulSoup(navegador.page_source, 'html.parser')

# Conteúdo HTML com índices de nome, ID e sinal de alerta;
usina_content = list_usinas.findAll('div',attrs={'class':'usina-info'})
alerta_content = list_usinas.findAll('div',attrs={'class':'usina-name'})
                                     
# Armazenamento de ID's das usinas 
id_usina = [link.find('a').get('href').split('=')[1] for link in usina_content] # <= Id's

# Armazenamento de nomes das usinas
nome_usina = [link.find('a').text.strip() for link in usina_content] # <= Nomes 
navigation_pag = list_usinas.findAll('li', attrs={'class': 'nav-item'})

# Armazenamento de alertas das usinas
alerta_usina = [link.find('img',attrs={'class':'status-img'}).get('src').split('/')[3][0] for link in alerta_content]

# Armazenamento da potência dos últimos 7 dias
pot_usina_html = list_usinas.findAll('div', attrs={'style': 'width: 12.8%; margin: auto 0px auto 0px; text-align: center; min-width: 90px'})
week_pot = [link.text.strip() for link in pot_usina_html] # <= Potencias

"""Como week_pot armazena a potência 'Ontem' e 'Últimos 7 dias', aqui é extraido somente os dados 
ímpares (que armazena 'Últimos 7 dias') """
pot_usina = [week_pot[num_potencia] for num_potencia in range(0, len(week_pot)) if num_potencia % 2]

# Loop para navegar nas páginas das usinas
for pagina in range(2, len(navigation_pag)):
    bot_on_click = navegador.find_element(By.LINK_TEXT, '{pg}'.format(pg = pagina)).click()
    sleep(10)
    list_usinas = BeautifulSoup(navegador.page_source, 'html.parser' )
    usina_content = list_usinas.findAll('div', attrs={'class':'usina-info'})
    alerta_content = list_usinas.findAll('div',attrs={'class':'usina-name'})
    pot_usina_html = list_usinas.findAll('div', attrs={'style': 'width: 12.8%; margin: auto 0px auto 0px; text-align: center; min-width: 90px'})
    week_pot = [link.text.strip() for link in pot_usina_html]
    for link in usina_content:
        id_usina.append(link.find('a').get('href').split('=')[1])
        nome_usina.append(link.find('a').text.strip())
    for link in alerta_content:
        alerta_usina.append(link.find('img',attrs={'class':'status-img'}).get('src').split('/')[3][0])
    for num_potencia in range(0, len(week_pot)):
        if num_potencia % 2:
            pot_usina.append(week_pot[num_potencia])

"""
alertas_usina = []
for alerta in alerta_usina:
    if alerta == 0:
        alertas_usina.append('Status não informado')
    elif alerta == 1:
        
# Transformando em DataFrame para manipulação dos dados    
data = {'Nome da usina':pd.Series(nome_usina), 'Geração Semanal':pd.Series(pot_usina), 'ID':pd.Series(id_usina)}
df = pd.DataFrame(data)
print(df)           

"""
